# HISR项目实验流程

## 项目概述
Hierarchical Invariant Sketch Resolution (HISR) - 基于多层不变学习的网络流量测量框架

## 目录结构
```
关键脚本实现示例（不一定完美复刻报告中的模型，仅参考）/
├── train_eval_hisr.py         # 统一训练评估入口
├── bucketize.py               # 逻辑分桶模块
├── encoder_bipartite.py       # 二分图编码器
├── decoder_prefix_tree.py     # 前缀树解码器
├── prefix.py                  # 层次化键空间表示
├── local_operator.py          # 桶本地算子提取
└── 其他支持文件...
```

## 1. 设置环境和导入模块

In [ ]:
import os
import sys
import subprocess
import json
from pathlib import Path


## 2. 设置项目路径

In [ ]:
# 设置路径
current_dir = Path.cwd()
hisr_scripts_dir = current_dir / "关键脚本实现示例（不一定完美复刻报告中的模型，仅参考）"
ucl_repo_dir = current_dir / "参考代码仓库" / "uclpe-sketch-master"

# 添加到Python路径
sys.path.insert(0, str(hisr_scripts_dir))
sys.path.insert(0, str(ucl_repo_dir))

print(f"HISR脚本目录: {hisr_scripts_dir}")
print(f"UCL代码仓库目录: {ucl_repo_dir}")
print("路径设置完成！")

## 3. 检查依赖和环境

In [ ]:
# 检查必要的包
def check_imports():
    required_packages = [
        'torch', 'numpy', 'tqdm', 'scipy', 'sklearn'
    ]
    
    missing_packages = []
    for package in required_packages:
        try:
            __import__(package)
            print(f"✅ {package} 已安装")
        except ImportError:
            missing_packages.append(package)
            print(f"❌ {package} 未安装")
    
    if missing_packages:
        print(f"\n需要安装的包: {', '.join(missing_packages)}")
        print("可以使用以下命令安装: pip install " + " ".join(missing_packages))
    else:
        print("\n✅ 所有依赖包已安装！")

check_imports()

## 4. 检查数据文件

In [ ]:
# 检查数据文件
def check_data_files():
    data_dir = ucl_repo_dir / "data"
    print(f"数据目录: {data_dir}")
    
    if data_dir.exists():
        dat_files = list(data_dir.glob("*.dat"))
        if dat_files:
            print("找到的数据文件:")
            for f in dat_files:
                print(f"  - {f.name}")
            return dat_files
        else:
            print("❌ 数据目录中没有找到 .dat 文件")
    else:
        print(f"❌ 数据目录不存在: {data_dir}")
    
    return []

data_files = check_data_files()

## 5. 配置文件生成

In [ ]:
# 生成简化的配置文件
config_content = {
    "data": "network",
    "data_path": "../参考代码仓库/uclpe-sketch-master/data/test-8s.dat" if data_files else None,
    "break_number": 100000,
    "bucket_len_stage1": 100,
    "bucket_len_stage2": 50,
    "bucket_len_stage3": 25,
    "topk_l1": 10,
    "topk_l2": 20,
    "num_envs": 3,
    "train_steps": 100,
    "lr": 0.001,
    "irm_lambda": 1.0,
    "inv_lambda": 0.1
}

config_file = hisr_scripts_dir / "demo_config.json"
if config_file.exists():
    print(f"配置文件已存在: {config_file}")
else:
    try:
        with open(config_file, 'w') as f:
            json.dump(config_content, f, indent=2)
        print(f"✅ 配置文件已创建: {config_file}")
    except Exception as e:
        print(f"❌ 创建配置文件失败: {e}")

## 6. 模块导入测试

In [ ]:
# 测试导入核心模块
def test_module_imports():
    modules_to_test = [
        "bucketize",
        "prefix", 
        "local_operator",
        "encoder_bipartite",
        "decoder_prefix_tree"
    ]
    
    print("测试模块导入:")
    for module_name in modules_to_test:
        try:
            module_path = hisr_scripts_dir / f"{module_name}.py"
            if module_path.exists():
                # 尝试导入
                import importlib.util
                spec = importlib.util.spec_from_file_location(module_name, module_path)
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)
                print(f"✅ {module_name}.py")
            else:
                print(f"❌ {module_name}.py 文件不存在")
        except Exception as e:
            print(f"❌ 导入 {module_name} 失败: {e}")

test_module_imports()

## 7. 运行HISR训练脚本（主流程）

In [ ]:
# 运行HISR训练脚本的函数
def run_hisr_training():
    script_path = hisr_scripts_dir / "train_eval_hisr.py"
    
    if not script_path.exists():
        print(f"❌ 主脚本不存在: {script_path}")
        return False
    
    # 基础的命令行参数
    cmd = [
        "python", str(script_path),
        "--data", "network",
        "--break_number", "100000"
    ]
    
    # 如果有数据文件，添加数据路径
    if data_files:
        # 使用第一个找到的数据文件
        data_file = data_files[0]
        cmd.extend(["--data_path", str(data_file)])
    else:
        print("⚠️  没有找到数据文件，使用合成数据模式")
        cmd.extend(["--data", "synthetic", "--skewness", "1.5"])
    
    print(f"执行命令: {' '.join(cmd)}")
    
    try:
        # 运行脚本
        result = subprocess.run(cmd, 
                              cwd=str(hisr_scripts_dir),
                              capture_output=True, 
                              text=True,
                              timeout=300)  # 5分钟超时
        
        print("=== 标准输出 ===")
        print(result.stdout)
        
        if result.stderr:
            print("=== 标准错误 ===")
            print(result.stderr)
        
        if result.returncode == 0:
            print("✅ HISR训练执行成功！")
            return True
        else:
            print(f"❌ HISR训练失败，返回码: {result.returncode}")
            return False
            
    except subprocess.TimeoutExpired:
        print("❌ 训练超时（超过5分钟）")
        return False
    except Exception as e:
        print(f"❌ 执行出错: {e}")
        return False

# 执行训练
print("开始执行HISR训练...")
training_success = run_hisr_training()

## 8. 验证实验结果

In [ ]:
# 检查训练结果
def check_training_results():
    # 检查可能的输出文件
    possible_outputs = [
        "checkpoints",
        "results.json",
        "training_log.txt",
        "model.pth"
    ]
    
    print("检查训练输出文件:")
    results_found = []
    for output in possible_outputs:
        output_path = hisr_scripts_dir / output
        if output_path.exists():
            results_found.append(output)
            print(f"✅ {output}")
        else:
            print(f"❌ {output}")
    
    return results_found

if training_success:
    outputs = check_training_results()
    if outputs:
        print(f"\n✅ 发现 {len(outputs)} 个输出文件")
    else:
        print("\n⚠️  未发现训练输出文件")
else:
    print("训练未成功执行，跳过结果检查")

## 9. 批量实验配置

In [ ]:
# 创建批量实验配置
experiment_configs = [
    {
        "name": "基线实验",
        "params": {
            "bucket_len_stage1": 100,
            "bucket_len_stage2": 50,
            "topk_l1": 10,
            "irm_lambda": 1.0
        }
    },
    {
        "name": "小桶配置",
        "params": {
            "bucket_len_stage1": 50,
            "bucket_len_stage2": 25,
            "topk_l1": 5,
            "irm_lambda": 2.0
        }
    },
    {
        "name": "大型配置",
        "params": {
            "bucket_len_stage1": 200,
            "bucket_len_stage2": 100,
            "topk_l1": 20,
            "irm_lambda": 0.5
        }
    }
]

print("配置的批量实验:")
for i, exp in enumerate(experiment_configs, 1):
    print(f"{i}. {exp['name']}: {exp['params']}")

## 10. 可视化支持（可选）

In [ ]:
# 简单的结果可视化函数
def setup_visualization():
    try:
        import matplotlib.pyplot as plt
        import pandas as pd
        
        print("✅ 可视化和分析工具已就绪")
        
        # 设置matplotlib中文字体（如果需要）
        plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
        plt.rcParams['axes.unicode_minus'] = False
        
        return True
    except ImportError as e:
        print(f"❌ 可视化包未安装: {e}")
        print("可以使用命令安装: pip install matplotlib pandas")
        return False

viz_ready = setup_visualization()

## 总结

这个Notebook提供了HISR项目的完整运行流程：

1. **环境设置** - 检查依赖和路径配置
2. **数据检查** - 验证数据文件是否存在
3. **模块测试** - 确保所有Python模块可以正常导入
4. **训练执行** - 运行HISR多环境训练流程
5. **结果验证** - 检查训练输出和模型文件
6. **批量实验** - 支持多种参数配置的对比实验

只需按顺序运行各个单元格即可完成整个实验流程！